In [249]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load CSV file as a DataFrame
prices = pd.read_csv('prices.csv')

# Load Excel file and its first sheet as a DataFrame
xls = pd.read_excel('room_types.xlsx', sheet_name=0)
room_types = pd.DataFrame(xls)

# Load TSV file as a DataFrame
reviews = pd.read_csv('reviews.tsv', sep='\t')

In [250]:
# Cleaning 'price' from prices
prices['price'] = prices['price'].str.replace(' dollars', '').astype(float)

In [251]:
round(prices['price'].mean(),2)

141.78

In [252]:
days_in_month = 30

prices['price_per_month'] = prices['price']*days_in_month

In [253]:
# Cleaning the case in room_type
room_types['room_type'] = room_types['room_type'].str.title()

In [254]:
# Convert the 'last_review' column to datetime format
reviews['last_review'] = pd.to_datetime(reviews['last_review'], format='%B %d %Y')

# Convert the date format to MM-DD-YYYY
reviews['last_review'] = reviews['last_review'].dt.strftime('%m-%d-%Y')

In [255]:
merge_df = pd.merge(reviews, room_types, on='listing_id')

In [256]:
df = pd.merge(merge_df, prices, on='listing_id')

In [257]:
# Extract the first word from each value in 'nbhood_full' column
df['borough'] = df['nbhood_full'].str.split().str.get(0)

# Remove the comma at the end of each word in 'borough' column
df['borough'] = df['borough'].str.rstrip(',')

# Remove the 'nbhood_full' column
df = df.drop('nbhood_full', axis=1)

In [258]:
# Remove rows with 0 values in the 'price' column
prices = prices[prices['price'] != 0]

In [259]:
# List for new column
conditions = [
    (df['price'] >= 0) & (df['price'] <= 69),
    (df['price'] >= 70) & (df['price'] <= 175),
    (df['price'] >= 176) & (df['price'] <= 350),
    (df['price'] > 350)
]
values = ['Budget', 'Average', 'Expensive', 'Extravagant']

# New column based on the conditions and values
df['price_range'] = np.select(conditions, values, default=None)

In [260]:
df

,listing_id,host_name,last_review,description,room_type,price,price_per_month,borough,price_range
0,2595,Jennifer,05-21-2019,Skylit Midtown Castle,Entire Home/Apt,225.0,6750.0,Manhattan,Expensive
1,3831,LisaRoxanne,07-05-2019,Cozy Entire Floor of Brownstone,Entire Home/Apt,89.0,2670.0,Brooklyn,Average
2,5099,Chris,06-22-2019,Large Cozy 1 BR Apartment In Midtown East,Entire Home/Apt,200.0,6000.0,Manhattan,Expensive
3,5178,Shunichi,06-24-2019,Large Furnished Room Near B'way,Private Room,79.0,2370.0,Manhattan,Average
4,5238,Ben,06-09-2019,Cute & Cozy Lower East Side 1 bdrm,Entire Home/Apt,150.0,4500.0,Manhattan,Average
...,...,...,...,...,...,...,...,...,...
25204,36425863,Rusaa,07-07-2019,Lovely Privet Bedroom with Privet Restroom,Private Room,129.0,3870.0,Manhattan,Average
25205,36427429,H Ai,07-07-2019,No.2 with queen size bed,Private Room,45.0,1350.0,Queens,Budget
25206,36438336,Ben,07-07-2019,Seas The Moment,Private Room,235.0,7050.0,Staten,Expensive
25207,36442252,Blaine,07-07-2019,1B-1B apartment near by Metro,Entire Home/Apt,100.0,3000.0,Bronx,Average


In [261]:
df.groupby('borough')['price_range'].value_counts().unstack().fillna(0)

price_range,Average,Budget,Expensive,Extravagant
borough,,,,
Bronx,286.0,382.0,25.0,5.0
Brooklyn,5534.0,3207.0,1466.0,259.0
Manhattan,5289.0,1150.0,3073.0,810.0
Queens,1505.0,1632.0,291.0,28.0
Staten,123.0,124.0,20.0,0.0


In [262]:
avg_price = round(prices['price'].mean(),2)

In [263]:
avg_price

141.82